In [1]:
%cd /opt/ml/project/team/code

/opt/ml/project/team/code


In [2]:
import torch
import json
import argparse
import os
import json
from attrdict import AttrDict
import sys
from torch.functional import split
import yaml
import torch
from torch.utils.data import DataLoader, SequentialSampler
from torch.cuda.amp import autocast
from tqdm.auto import tqdm
import copy

from prepare import get_data, get_stuff, get_model
import parser_maker
from data_utils import split_slot, convert_state_dict, WOSDataset

from training_recorder import RunningLossRecorder
import glob
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Optional, Union
import pickle

In [3]:
task_dir = '/opt/ml/project/team/code/results/random_1125_1'

root_name = '/opt/ml/input/data/tmp/'
files = glob.glob(f'{root_name}/need_overgen*')
output_dir = '/opt/ml/input/data/tmp'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# conf=dict()
# with open(config_root) as f:
#     conf = yaml.load(f, Loader=yaml.FullLoader)


# eval_data = json.load(open(f"/opt/ml/input/data/eval_dataset/eval_dials.json", "r"))

config = json.load(open(f"{task_dir}/exp_config.json", "r"))

# print(config)
config = argparse.Namespace(**config)
_, slot_meta, ontology = get_data(config)

config.device = device

config.use_cache_examples2features = False
tokenizer, processor, _, _ = get_stuff(config,
             None, None, slot_meta, ontology)

# eval_data = WOSDataset(eval_features)
# eval_sampler = SequentialSampler(eval_data)
# eval_loader = DataLoader(
#     eval_data,
#     batch_size=8,
#     sampler=eval_sampler,
#     collate_fn=processor.collate_fn,
# )
# print("# eval:", len(eval_data))

# print(slot_meta)

model =  get_model(config, tokenizer, ontology, slot_meta)


ckpt = torch.load(f"{task_dir}/model-best.bin", map_location="cpu")
# ckpt = torch.load("/opt/ml/gyujins_file/model-best.bin", map_location="cpu")

model.load_state_dict(ckpt)
model.to(device)
print('done')

using train: train_dials.json



done


In [5]:


data = []

for file in files:
#     print(file)
    cur_batch = pickle.load(open(file, 'rb'))
#     print(cur_batch[0])
    data.extend(cur_batch)

In [6]:


@dataclass
class DSTInputExample:
    guid: str
    context_turns: List[str]
    current_turn: List[str]
    label: Optional[List[str]] = None
    before_label: Optional[List[str]] = None

    def to_dict(self):
        return dataclasses.asdict(self)

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2) + "\n"

In [7]:
def postprocess_state(state):
    for i, s in enumerate(state):
        s = s.replace(" : ", ":")
        s = s.replace(" & ", "&")
        s = s.replace(" = ", "=")
        s = s.replace("( ", "(")
        s = s.replace(" )", ")")
        state[i] = s.replace(" , ", ", ")
    return state

In [8]:
examples = []
for idx, datum in enumerate(data):
    examples.append(
            DSTInputExample(
                guid=f"{datum.guid}-{datum.turn}",
                context_turns=["", ""],
                current_turn=[datum.sys, datum.text],  # TRADE,SUMBT 다름 유의
                label=datum.state, 
                before_label={}, # This was for som_dst
            )
    )

In [9]:
features = processor.convert_examples_to_features(examples)

In [10]:
ds = WOSDataset(features)
sampler = SequentialSampler(ds)
eval_loader = DataLoader(
    ds,
    batch_size=8,
    sampler=sampler,
    collate_fn=processor.collate_fn,
)

In [15]:
input_ids.shape, segment_ids.shape, input_masks.dtype

(torch.int64, torch.int64, torch.bool)

In [ ]:
gating_ids.dtype

In [11]:
use_amp = False

model.eval()
predictions = {}
pbar = tqdm(eval_loader, total=len(eval_loader), file=sys.stdout)

for batch in pbar:
    input_ids, segment_ids, input_masks, gating_ids, target_ids, guids = [
        b.to(device) if not isinstance(b, list) else b for b in batch
    ]
#     print(input_ids.shape)

    with torch.no_grad():
        with autocast(enabled=use_amp):
            o, g = model(input_ids, segment_ids, input_masks, 9)

        _, generated_ids = o.max(-1)
        _, gated_ids = g.max(-1)


    for guid, gate, gen in zip(guids, gated_ids.tolist(), generated_ids.tolist()):
        prediction = processor.recover_state(gate, gen)
        prediction = postprocess_state(prediction)
        predictions[guid] = prediction
pbar.close()


torch.Size([8, 68])
torch.Size([8, 64])
torch.Size([8, 66])
torch.Size([8, 59])
torch.Size([8, 109])
torch.Size([8, 69])
torch.Size([8, 49])
torch.Size([8, 54])
torch.Size([8, 86])
torch.Size([8, 46])
torch.Size([8, 72])
torch.Size([8, 69])
torch.Size([8, 81])
torch.Size([8, 53])
torch.Size([8, 58])
torch.Size([8, 90])
torch.Size([8, 68])
torch.Size([8, 91])
torch.Size([8, 54])
torch.Size([8, 64])
torch.Size([8, 78])
torch.Size([8, 79])
torch.Size([8, 74])
torch.Size([8, 84])
torch.Size([8, 65])
torch.Size([8, 80])
torch.Size([8, 67])
torch.Size([8, 76])
torch.Size([8, 72])
torch.Size([8, 69])
torch.Size([8, 58])
torch.Size([8, 68])
torch.Size([8, 120])
torch.Size([8, 118])
torch.Size([8, 87])
torch.Size([8, 104])
torch.Size([8, 100])
torch.Size([8, 70])
torch.Size([8, 92])
torch.Size([8, 47])
torch.Size([8, 82])
torch.Size([8, 74])
torch.Size([8, 63])
torch.Size([8, 74])
torch.Size([8, 70])
torch.Size([8, 50])
torch.Size([8, 75])
torch.Size([8, 59])
torch.Size([8, 58])
torch.Size([8, 

KeyboardInterrupt: 

In [ ]:
good = []
bad = []
good_idxs = []
for i, example in enumerate(examples):
    assert example.guid == f'{data[i].guid}-{data[i].turn}'
    guid = example.guid

    answer = convert_state_dict(example.label)
    pred = convert_state_dict(predictions[guid])
    if set(pred).issubset(set(answer)):
        good_idxs.append(i)
        good.append(
            AttrDict(
                example=example,
                prediction=predictions[guid],
            )
        )
    else:
        bad.append(
            AttrDict(
                example=example,
                prediction=predictions[guid],
            )
        )
    
len(good), len(bad)

In [ ]:
idx = 0

In [ ]:
print(f'SYS: {bad[idx].example.current_turn[0]}')
print(f'USR: {bad[idx].example.current_turn[1]}')
print(f' ANS: {sorted(bad[idx].example.label)}')
print(f'PRED: {sorted(bad[idx].prediction)}')
print(bad[idx].example.guid)
idx += 1

In [ ]:
idx = 0

In [ ]:
print(f'SYS: {good[idx].example.current_turn[0]}')
print(f'USR: {good[idx].example.current_turn[1]}')
print(f' ANS: {sorted(good[idx].example.label)}')
print(f'PRED: {sorted(good[idx].prediction)}')
idx += 1

In [ ]:
good[0]

In [ ]:
# diff_good = []
# for x in good:
#     if set(x.example.label) != set(x.prediction):
#         diff_good.append(x)

In [ ]:
idx = 0

In [ ]:
diff_good = []
diff_idxs = []
for i, x in enumerate(good):
    if set(x.example.label) != set(x.prediction):
        diff_good.append(x)
        diff_idxs.append(good_idxs[i])

In [ ]:
for i, diff_idx in enumerate(diff_idxs):
    assert diff_good[i].example.guid == f'{data[diff_idx].guid}-{data[diff_idx].turn}'

In [ ]:
saving = defaultdict(list)
for diff_idx in diff_idxs:
    datum = data[diff_idx]
    turn_stuff = {
        'text':datum.text,
        'state':list(datum.state),
    }
    
    guid_stuff = {
        datum.turn: turn_stuff
    }
    
    saving[datum.guid].append(
        guid_stuff
    )

In [ ]:
idx = 0
while True:
    name = f'{output_dir}/final_{idx}.pkl'
    if not os.path.exists(name):
        break
    idx += 1

name = f'{output_dir}/final_size_{len(diff_idxs)}_{idx}.pkl' 
pickle.dump(saving, open(name, 'wb'))

In [ ]:
idx = 0

In [ ]:
print(f'SYS: {diff_good[idx].example.current_turn[0]}')
print(f'USR: {diff_good[idx].example.current_turn[1]}')
print(f' ANS: {sorted(diff_good[idx].example.label)}')
print(f'PRED: {sorted(diff_good[idx].prediction)}')
idx += 1